# Lecture 15 

Assembling dynamics through the E-L equations

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi, lambdify
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

from scipy.integrate import solve_ivp

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
    


In [2]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 m_1,
 m_2,
 g) = symbols("""t, 
                         theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3
                         m_1
                         m_2
                         g""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [4]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)


Let's define our homogeneous transformation matrix that applies a rotation and a translation to vectors and matrices

In [5]:
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK1

Matrix([
[l_1*cos(theta_1(t))],
[l_1*sin(theta_1(t))]])

In [6]:
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK2

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))]])

In [7]:
v1 = diff(FK1,t)
KE1 = (1/2)*m_1*(v1.T * v1)[0]
KE1

0.5*m_1*(l_1**2*sin(theta_1(t))**2*Derivative(theta_1(t), t)**2 + l_1**2*cos(theta_1(t))**2*Derivative(theta_1(t), t)**2)

In [8]:
v2 = diff(FK2,t)
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE2

0.5*m_2*((-l_1*sin(theta_1(t))*Derivative(theta_1(t), t) - l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)))**2 + (l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)))**2)

# Assemble Lagrangian

In [9]:
T = KE1 + KE2
V = m_1*g*FK1[1] + m_2*g*FK2[1]

L = T - V
L = simplify(L)
L

-g*l_1*m_1*sin(theta_1(t)) - g*m_2*(l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))) + 0.5*l_1**2*m_1*Derivative(theta_1(t), t)**2 + 0.5*m_2*(l_1**2*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_1(t), t)**2 + 2*l_2**2*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_2(t), t)**2)

# Derive EOMs from Euler-Lagrange equations

In [10]:
EOM_theta_1 = diff(diff(L, Derivative(theta_1, t)), t) - diff(L, theta_1)
EOM_theta_1 = nsimplify(EOM_theta_1)
EOM_theta_1

g*l_1*m_1*cos(theta_1(t)) + g*m_2*(l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))) + l_1**2*m_1*Derivative(theta_1(t), (t, 2)) + m_2*(2*l_1**2*Derivative(theta_1(t), (t, 2)) - 4*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 + 4*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_2(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2

In [11]:
EOM_theta_2 = diff(diff(L, Derivative(theta_2, t)), t) - diff(L, theta_2)
EOM_theta_2 = nsimplify(EOM_theta_2)
EOM_theta_2

g*l_2*m_2*cos(theta_1(t) + theta_2(t)) - m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2 - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t))/2 + m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2

# Assemble the mass matrix

In [12]:
mass = sp.symarray('',(2,2))

mass[0,0] = EOM_theta_1.expand().coeff(Derivative(theta_1,t,t))

mass[0,1] = EOM_theta_1.expand().coeff(Derivative(theta_2,t,t))
mass[1,0] = mass[0,1]

mass[1,1] = EOM_theta_2.expand().coeff(Derivative(theta_2,t,t))

mass = Matrix(mass)

In [13]:
mass = simplify(mass)
mass

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + 2*l_1*l_2*m_2*cos(theta_2(t)) + l_2**2*m_2, l_2*m_2*(l_1*cos(theta_2(t)) + l_2)],
[                                 l_2*m_2*(l_1*cos(theta_2(t)) + l_2),                          l_2**2*m_2]])

In [14]:
accel_state_vector = Matrix([[Derivative(theta_1, t, t)], [Derivative(theta_2, t, t)]])

EOM1_leftover = simplify(EOM_theta_1.expand() - (mass[0,:]*accel_state_vector)[0])
EOM2_leftover = simplify(EOM_theta_2.expand() - (mass[1,:]*accel_state_vector)[0])


In [15]:
EOM1_leftover

g*l_1*m_1*cos(theta_1(t)) + g*l_1*m_2*cos(theta_1(t)) + g*l_2*m_2*cos(theta_1(t) + theta_2(t)) - 2*l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2

In [16]:
EOM2_leftover

l_2*m_2*(g*cos(theta_1(t) + theta_2(t)) + l_1*sin(theta_2(t))*Derivative(theta_1(t), t)**2)

Grab just the gravity vectory terms

In [17]:
G_1 = EOM1_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0)])
G_1

g*l_1*m_1*cos(theta_1(t)) + g*l_1*m_2*cos(theta_1(t)) + g*l_2*m_2*cos(theta_1(t) + theta_2(t))

In [18]:
G_2 = EOM2_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0)])
G_2

g*l_2*m_2*cos(theta_1(t) + theta_2(t))

Grab the left over Coriolis terms

In [19]:
C_1 = EOM1_leftover - G_1
C_1

-2*l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2

In [20]:
C_2 = simplify(EOM2_leftover - G_2)
C_2

l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2

Check that we got everything right!

In [21]:
simplify(Matrix([[EOM_theta_1], [EOM_theta_2]]) - (mass*accel_state_vector + Matrix([[C_1], [C_2]]) + Matrix([[G_1], [G_2]])))

Matrix([
[0],
[0]])

## Setup a simulation of the system

In [22]:
M =   lambdify((theta_1, theta_2, Derivative(theta_1, t), Derivative(theta_2, t), l_1, l_2, m_1, m_2), mass)
EOM = lambdify((theta_1, theta_2, Derivative(theta_1, t), Derivative(theta_2, t), l_1, l_2, m_1, m_2), 
               Matrix([[EOM1_leftover.subs([(g, 9.8)])], [EOM2_leftover.subs([(g, 9.8)])]]))

In [23]:
M(0,0, 0, 0, 1, 1, 1, 1)

array([[5., 2.],
       [2., 1.]])

In [24]:
EOM(0, np.pi/2, 10, 10, 1, 1, 1, 1)

array([[-280.4],
       [ 100. ]])

In [25]:
def dynamics(t, state, l_1, l_2, m_1, m_2):
    theta_1, theta_2, dtheta_1, dtheta_2 = state
    
    dydt = np.linalg.inv(M(theta_1, theta_2, dtheta_1, dtheta_2, l_1, l_2, m_1, m_2))@(-EOM(theta_1, theta_2, dtheta_1, dtheta_2, l_1, l_2, m_1, m_2))

    return [dtheta_1, dtheta_2, dydt[0][0], dydt[1][0]]

In [26]:
dynamics(t, (0, np.pi/2, 10, 10), 1,1,1,1)

[10, 10, 190.2, -290.19999999999993]

In [29]:
l1 = 1
l2 = 2
m1 = 1
m2 = 2


t_end = 30
dt = 0.001
time = np.linspace(0,t_end,int(t_end/dt))

sol = solve_ivp(lambda t, y: dynamics(t, y, l1, l2, m1, m2),
                [0,t_end], [-np.pi/4, 0.0, 0.00, 0.0], 
                t_eval = time, 
                rtol=1e-8, atol = 1e-8)


plt.clf()
plt.plot(sol.t, sol.y[0,:])
# plt.ylabel('$x$')
# plt.xlabel('time')

Make an animation

In [30]:
skip = 10

theta1 = sol.y[0,0:-1:skip]
theta2 = sol.y[1,0:-1:skip]

for kk, (q1, q2) in enumerate(zip(theta1, theta2)):
    plt.cla()
    # print(q1)
    
    plt.plot([0,l1*np.cos(q1)], [0, l1*np.sin(q1)], 'ko-')
    
    plt.plot([l1*np.cos(q1), l1*np.cos(q1) + l2*np.cos(q1 + q2)], [l1*np.sin(q1), l1*np.sin(q1)  + l2*np.sin(q1 + q2)], 'ko-')
    plt.axis([-4,4,-4,4])

    plt.pause(0.00001)


KeyboardInterrupt: 